In [27]:
# Importing libraries
import pandas as pd
import numpy as np

In [28]:
# Loading datasets
prices = pd.read_csv('/Users/macpro/Downloads/Data Wrangling Practise/prices.csv')
sales = pd.read_csv('/Users/macpro/Downloads/Data Wrangling Practise/sales.csv')

In [29]:
# Reading the first 5 rows of the dataset
prices.head()

,product_id,old_price,new_price,updated_at
0,64,270000,239000,9/10/18 16:37
1,3954203,60000,64000,9/11/18 11:54
2,3954203,60500,57500,9/17/18 22:59
3,3954203,64000,60500,9/15/18 3:49
4,3954203,68800,60000,9/10/18 16:32


In [30]:
# Reading the first 5 rows of the dataset
sales.head()

,product_id,ordered_at,quantity_ordered
0,3998909,9/18/18 17:51,1
1,3998909,9/18/18 12:52,1
2,3998909,9/18/18 11:33,1
3,3998909,9/18/18 18:47,1
4,3998909,9/18/18 17:36,1


In [31]:
# Merging 2 datasets
merged = sales.merge(prices,on='product_id',how='inner')

In [32]:
# Converting the 'ordered_at' and 'updated_at' columns to datetime format
merged['ordered_at'] = pd.to_datetime(merged['ordered_at'])
merged['updated_at'] = pd.to_datetime(merged['updated_at'])

In [33]:
# Calculating the time difference between 'ordered_at' and 'updated_at' and storing it in the 'ordered_vs_updated' column:
merged['ordered_vs_updated'] = merged['ordered_at'] - merged['updated_at']

In [34]:
# alculating the absolute value of the 'ordered_vs_updated' column and storing it in the 'ordered_vs_updated_abs' column
merged['ordered_vs_updated_abs'] = merged['ordered_vs_updated'].abs()

In [35]:
# Filtering the merged DataFrame to keep only rows where 'ordered_vs_updated_abs' is equal to the minimum value for each unique combination of 'product_id' and 'ordered_at'
merged=merged[merged['ordered_vs_updated_abs'] == merged.groupby(['product_id', 'ordered_at'])['ordered_vs_updated_abs'].transform('min')]

In [36]:
# Determining the 'purchase_price' based on the condition that if 'ordered_vs_updated' is greater than or equal to zero, use the 'new_price' column; otherwise, use the 'old_price' column
merged['purchase_price'] = np.where(merged['ordered_vs_updated'] >= pd.to_timedelta('0S'), merged['new_price'], merged['old_price'])

In [37]:
# Grouping the data by 'product_id' and 'purchase_price' and calculating the sum of 'quantity_ordered' for each group
final_result = merged.groupby(['product_id', 'purchase_price'], as_index=False)['quantity_ordered'].sum()

In [38]:
# Calculating the total revenue for each group by multiplying 'purchase_price' with 'quantity_ordered'
final_result['total_revenue'] = final_result['purchase_price'] * final_result['quantity_ordered']

In [39]:
# Appending a row to the DataFrame with the sum of all the numeric columns, which represents the total revenue
final_result.loc['Total'] = final_result.sum(numeric_only=True, axis=0)

In [40]:
# Setting the 'product_id' and 'purchase_price' values of the last row to an empty string (for aesthetic purposes)
final_result.loc[final_result.index[-1], ['product_id', 'purchase_price']] = ''

In [41]:
# Printing the final result DataFrame
print(final_result)

      product_id purchase_price  quantity_ordered  total_revenue
0             64         239000                 4         956000
1        3954203          57500                 1          57500
2        3954203          60000                 3         180000
3        3954203          64000                10         640000
4        3998909          15500                 1          15500
5        3998909          16500                14         231000
6        3998909          17000                 2          34000
7        4085861          52000                20        1040000
8        4085861          53500                40        2140000
9        4085861          58000                38        2204000
10       4085861          60000                 3         180000
11       4085861          62500                29        1812500
12       4085861          67000                13         871000
Total                                         178       10361500
